# Material do curso de nivelamento em Deep Learning do professor Edson M. Takashi (FACOM - UFMS)

In [ ]:
import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
import pandas as pd
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import os

%cd /content/drive/MyDrive/doutorado/topicos nlp/word2vec
#!ls

/content/drive/MyDrive/doutorado/topicos nlp/word2vec


In [ ]:
body = pickle.load(open("body.p",'br'))

In [ ]:
title = pickle.load(open("title.p",'br'))

In [ ]:
len(body),len(title)

(1336, 1336)

In [ ]:
scorpus = []
for inst in body+title:
    scorpus += sent_tokenize(inst)
fcorpus = []
for inst in scorpus:
    fcorpus += [word_tokenize(inst)]

In [ ]:
!gdown https://drive.google.com/uc?id=1-XXx9A7EMCfP5LLLPLk6HIJbaJLDGvNa

Downloading...
From: https://drive.google.com/uc?id=1-XXx9A7EMCfP5LLLPLk6HIJbaJLDGvNa
To: /content/drive/My Drive/doutorado/topicos nlp/word2vec/dataset.p
100% 20.7M/20.7M [00:00<00:00, 127MB/s]


In [ ]:
corpus = pickle.load(open('dataset.p','rb'))
scorpus = []
for inst in corpus:
    scorpus += sent_tokenize(inst['body'])
    scorpus += sent_tokenize(inst['title'])

fcorpus = []
for inst in scorpus:
    fcorpus += [word_tokenize(inst)]

## Tokens especiais (twitter)

In [ ]:
!pip install ekphrasis

     |████████████████████████████████| 80 kB 3.7 MB/s 
     |████████████████████████████████| 214 kB 18.8 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-py3-none-any.whl size=82842 sha256=3f6fbe079f9ebd4298994ad582a16855a27318794a2c320ebe148d553e6d0c85
  Stored in directory: /root/.cache/pip/wheels/f7/ec/0d/12659e32faf780546945d0120f2c8410eb3efb7426731da88f
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=09a894db406b6b28a5470daa3d4665fcef388de8fd8795b3a124c868a5ae1c8e
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ekphrasis ftfy


In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
text_processor.pre_process_doc("Tô que nem o MC Doni, virei fábrica de hit FogoFogoFogo")

['tô',
 'que',
 'nem',
 'o',
 'mc',
 'doni',
 ',',
 'virei',
 'fábrica',
 'de',
 'hit',
 'fogofogofogo']

### Fasttext

In [ ]:
from gensim.models import FastText

In [ ]:
sentences = [['cat', 'say', 'meow'], ['dog', 'say', 'woof', 'faca', 'comp']]
model = FastText(sentences, min_count=1)

In [ ]:
model.most_similar('facom')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('faca', 0.5361242294311523),
 ('comp', 0.25643062591552734),
 ('dog', 0.20705343782901764),
 ('meow', 0.09984646737575531),
 ('say', 0.056755296885967255),
 ('cat', -0.06016944348812103),
 ('woof', -0.06544774025678635)]

## Byte-Pair Encoding

In [ ]:
!pip install tokenizers

     |████████████████████████████████| 3.3 MB 5.0 MB/s 


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE())

from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

In [ ]:
from tqdm.auto import tqdm  

text_data = []
file_count = 0

for sample in tqdm(scorpus):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample.replace('\n', '\s')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'./text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we may have leftover samples, we save those now too
with open(f'./text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/9590 [00:00<?, ?it/s]

In [ ]:
from pathlib import Path
paths = [str(x) for x in Path('./').glob('**/*.txt')]
paths[:5]

['text_0.txt', 'text_1.txt']

In [ ]:
from tokenizers.trainers import BpeTrainer

In [ ]:
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [ ]:
tokenizer.train(paths, trainer=trainer)

In [ ]:
enc= tokenizer.encode('Cresce a expectativa de que o ajuste na taxa Selic, hoje em 6,25%, seja de 1,5 ponto percentual e algumas casas chegam a prever um aumento ainda maior')

In [ ]:
enc.tokens

['Cresce',
 'a',
 'expectativa',
 'de',
 'que',
 'o',
 'ajuste',
 'na',
 'taxa',
 'Selic',
 ',',
 'hoje',
 'em',
 '6',
 ',',
 '25',
 '%,',
 'seja',
 'de',
 '1',
 ',',
 '5',
 'ponto',
 'percentual',
 'e',
 'algumas',
 'casas',
 'chegam',
 'a',
 'prever',
 'um',
 'aumento',
 'ainda',
 'maior']

## WordPiece

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt

--2021-10-27 21:05:27--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.16.102
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.16.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231508 (226K) [text/plain]
Saving to: ‘bert-base-uncased-vocab.txt’

bert-base-uncased-v 100%[===================>] 226.08K  --.-KB/s    in 0.07s   

2021-10-27 21:05:28 (3.11 MB/s) - ‘bert-base-uncased-vocab.txt’ saved [231508/231508]



In [ ]:
from tokenizers import BertWordPieceTokenizer

sequence = "How are you Tokenizer 😁 ?"
tokenizerBW = BertWordPieceTokenizer("bert-base-uncased-vocab.txt")
tokenized_sequenceBW = tokenizerBW.encode(sequence)
print(tokenized_sequenceBW)
print(type(tokenized_sequenceBW))
print(tokenized_sequenceBW.ids)
print(tokenized_sequenceBW.tokens)

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
<class 'tokenizers.Encoding'>
[101, 2129, 2024, 2017, 19204, 17629, 100, 1029, 102]
['[CLS]', 'how', 'are', 'you', 'token', '##izer', '[UNK]', '?', '[SEP]']


In [ ]:
tokenized_sequenceBW = tokenizerBW.encode("surfboarded surf surfing surfed")
print(tokenized_sequenceBW)
print(type(tokenized_sequenceBW))
print(tokenized_sequenceBW.ids)
print(tokenized_sequenceBW.tokens)

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
<class 'tokenizers.Encoding'>
[101, 14175, 6277, 2098, 14175, 19967, 14175, 2098, 102]
['[CLS]', 'surf', '##board', '##ed', 'surf', 'surfing', 'surf', '##ed', '[SEP]']
